In [3]:
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd
import math
import os
import random
from scipy import interpolate
import copy
from multiprocessing import Pool
import PyMieScatt as ps
import tkinter
from tkinter import ttk

#0:transmission、1:forward scattering 2:sideward scattering 3:backward scattering 4:absorption
TRANSMISSION = 0
FORWARD_SCATTERING = 1
SIDEWARD_SCATTERING = 2
SIDEWARD_SCATTERING_X = 2
SIDEWARD_SCATTERING_Y = 2
BACKWARD_SCATTERING = 3
ABSORPTION = 4
IN_SYSTEM = 9

MAX_PARICLE_TYPES = 3
RANDOM_SEED = 1

WID =  5

ANGLE_ARR_NUMBER = 1801
ANGLE_ARR = np.linspace(0,math.pi,ANGLE_ARR_NUMBER)

INITIAL_SETTING  = pd.read_csv("setting/initial_setting.csv",index_col = 0)

def save_initial_setting():
    INITIAL_SETTING["NUMBER_OF_TYPES"] = setting.particle_types.get()
    INITIAL_SETTING["MIN_WAVE"] = round_sig(float(setting.min_wave.get()))
    INITIAL_SETTING["MAX_WAVE"] = round_sig(float(setting.max_wave.get()))
    INITIAL_SETTING["INTERVAL_WAVE"] = round_sig(float(setting.interval_wave.get()))
    INITIAL_SETTING["NUMBER_OF_INCIDENT_PHOTONS"] = int(setting.incident_photons.get()) 
    INITIAL_SETTING["BLOCK_X"] = setting.block_x.get()
    INITIAL_SETTING["BLOCK_Y"] = setting.block_y.get()
    INITIAL_SETTING["BLOCK_Z"] = setting.block_z.get()
    INITIAL_SETTING["HOST_REFRACTIVE_INDEX"] = float(setting.host_refractive_index.get())
    INITIAL_SETTING["PERIODIC_BOUNDARY_MODE"] = str(setting.PERIODIC_BOUNDARY_MODE.get())
    INITIAL_SETTING["LOG_MODE"] = str(setting.LOG_MODE.get())
    for num in range(MAX_PARICLE_TYPES):
        INITIAL_SETTING["NO"+str(num+1)+"_HAVESHELL"] = str(particles[num].haveshell.get())
        INITIAL_SETTING["NO"+str(num+1)+"_CORE_MATERIAL"] =  material_files.index(particles[num].core_material_combo.get())
        INITIAL_SETTING["NO"+str(num+1)+"_SHELL_MATERIAL"] = material_files.index(particles[num].shell_material_combo.get())
        INITIAL_SETTING["NO"+str(num+1)+"_IMPORT_FILE"] = custom_files.index(particles[num].import_file_combo.get())
        INITIAL_SETTING["NO"+str(num+1)+"_CUSTOM_VALUE1"] = particles[num].custom_value1.get()
        INITIAL_SETTING["NO"+str(num+1)+"_CUSTOM_VALUE2"] = particles[num].custom_value2.get()
        INITIAL_SETTING["NO"+str(num+1)+"_CUSTOM_VALUE3"] = particles[num].custom_value3.get()
        INITIAL_SETTING["NO"+str(num+1)+"_CUSTOM_VALUE4"] = particles[num].custom_value4.get()
        INITIAL_SETTING["NO"+str(num+1)+"_CONCENTRATION"] = particles[num].concentration.get()
        INITIAL_SETTING["NO"+str(num+1)+"_CORE_DIAMETER"] = particles[num].core_diameter.get()
        INITIAL_SETTING["NO"+str(num+1)+"_SHELL_THICKNESS"]  = particles[num].shell_thickness.get()
    INITIAL_SETTING.to_csv("setting/initial_setting.csv")
    print("SAVED SETTING")
    
def round_sig(x, sig=5):
    if(x==0):
        return 0
    return round(x, sig-int(math.floor(math.log10(abs(x))))-1)

class Setting(tkinter.Frame):
    start_row = None   
    num = None
    #define UI
    def __init__(self, master=None):
        super().__init__(master)
        
    def particle_types(self):
        first_label = tkinter.Label(text="##################################Particle Information##################################")
        first_label.grid(row=self.start_row, column=0,columnspan = 5, padx=5,)
        labeler(self.start_row+1,0,1,"  ")
        particle_types_label = tkinter.Label(text="number of types of particles")
        particle_types_label.grid(row=self.start_row+2, column=1, padx=10,)
        self.particle_types = tkinter.Entry(width=WID)
        self.particle_types.insert(tkinter.END,INITIAL_SETTING["NUMBER_OF_TYPES"].values[0])
        self.particle_types.grid(row=self.start_row+2, column=2)

    def simulation(self):
        second_label = tkinter.Label(text="##################################Simulation Settings##################################")
        second_label.grid(row=self.start_row, column=0,columnspan = 5, padx=5,pady = 10)

        #wavelength
        min_wave_label = tkinter.Label(text="min wave (nm)").grid(row=self.start_row+1, column=1, padx=5,)
        self.min_wave = tkinter.Entry(width=WID)
        self.min_wave.insert(tkinter.END,INITIAL_SETTING["MIN_WAVE"].values[0])
        self.min_wave.grid(row=self.start_row+1, column=2)

        max_wave_label = tkinter.Label(text="max wave (nm)").grid(row=self.start_row+1, column=3, padx=5,)
        self.max_wave = tkinter.Entry(width=WID)
        self.max_wave.insert(tkinter.END,INITIAL_SETTING["MAX_WAVE"].values[0])
        self.max_wave.grid(row=self.start_row+1, column=4)

        interval_wave_label = tkinter.Label(text="interval wave (nm)").grid(row=self.start_row+1, column=5, padx=5,)
        self.interval_wave = tkinter.Entry(width=WID)
        self.interval_wave.insert(tkinter.END,INITIAL_SETTING["INTERVAL_WAVE"].values[0])
        self.interval_wave.grid(row=self.start_row+1, column=6)

        #number of incident photons
        incident_photons_label = tkinter.Label(text="number of incident photons")
        incident_photons_label.grid(row=self.start_row+2, column=1, padx=10,)
        self.incident_photons = tkinter.Entry(width=WID)
        self.incident_photons.insert(tkinter.END,INITIAL_SETTING["NUMBER_OF_INCIDENT_PHOTONS"].values[0])
        self.incident_photons.grid(row=self.start_row+2, column=2)

        #system
        block_x_label = tkinter.Label(text="x (um)").grid(row=self.start_row+3, column=1, padx=10,)
        self.block_x = tkinter.Entry(width=WID)
        self.block_x.insert(tkinter.END,INITIAL_SETTING["BLOCK_X"].values[0])
        self.block_x.grid(row=self.start_row+3, column=2)

        block_y_label = tkinter.Label(text="y (um)").grid(row=self.start_row+3, column=3, padx=10,)
        self.block_y = tkinter.Entry(width=WID)
        self.block_y.grid(row=self.start_row+3, column=4)
        self.block_y.insert(tkinter.END,INITIAL_SETTING["BLOCK_Y"].values[0])

        block_z_label = tkinter.Label(text="z (um)").grid(row=self.start_row+3, column=5, padx=10,)
        self.block_z = tkinter.Entry(width=WID)
        self.block_z.grid(row=self.start_row+3, column=6)
        self.block_z.insert(tkinter.END,INITIAL_SETTING["BLOCK_Z"].values[0])

        #host refractive index
        host_refractive_index_label = tkinter.Label(text="host refractive index")
        host_refractive_index_label.grid(row=self.start_row+4, column=1, padx=10,)
        self.host_refractive_index = tkinter.Entry(width=WID)
        self.host_refractive_index.insert(tkinter.END,INITIAL_SETTING["HOST_REFRACTIVE_INDEX"].values[0])
        self.host_refractive_index.grid(row=self.start_row+4, column=2)

        #PERIODIC_BOUNDARY
        self.PERIODIC_BOUNDARY_label = tkinter.Label(text="PERIODIC BOUNDARY MODE").grid(row=self.start_row+5, column=0,columnspan = 3, padx=5,)
        self.PERIODIC_BOUNDARY_MODE = tkinter.BooleanVar()
        self.PERIODIC_BOUNDARY_MODE.set(bool(INITIAL_SETTING["PERIODIC_BOUNDARY_MODE"].values[0]))
        self.PERIODIC_BOUNDARY_MODE_button = tkinter.Checkbutton(text=u"", variable=self.PERIODIC_BOUNDARY_MODE)
        self.PERIODIC_BOUNDARY_MODE_button.grid(row=self.start_row+5, column=3)

        #LOG_MODE
        LOG_MODE_label = tkinter.Label(text="OUTPUT LOG FILE").grid(row=self.start_row+6, column=0, padx=5,columnspan = 3)
        self.LOG_MODE = tkinter.BooleanVar()
        self.LOG_MODE.set(bool(INITIAL_SETTING["LOG_MODE"].values[0]))
        self.LOG_MODE_button = tkinter.Checkbutton(text=u"", variable=self.LOG_MODE)
        self.LOG_MODE_button.grid(row=self.start_row+6, column=3)

        none_label2 = tkinter.Label(text="").grid(row=self.start_row+7, column=0,columnspan = 5, padx=5,pady = 10)

        third_label = tkinter.Label(text="##################################Output Files##################################")
        third_label.grid(row=self.start_row+8, column=0,columnspan = 5, padx=5,pady = 10)

        #Start and Stop
        self.start_button = tkinter.Button(text="Start",command = start)
        self.start_button.grid(row=self.start_row+9, column=1)
        self.end_button = tkinter.Button(text = "Quit", command = quit)
        self.end_button.grid(row=self.start_row+9, column=4)


class ParticleInfo(tkinter.Frame):
    start_row = None   
    num = None
    #defineUI
    def __init__(self, master=None):
        super().__init__(master)
        
    #Define widgets
    def define_widget(self):
        self.haveshell = tkinter.BooleanVar()
        self.haveshell.set(bool(INITIAL_SETTING["NO"+str(self.num)+"_HAVESHELL"].values[0]))
        
        self.core_only_radio  = tkinter.Radiobutton(root,text='Core-Only ',variable=self.haveshell, value=False,command = self.delete_core_shell_label)
        self.core_shell_radio = tkinter.Radiobutton(root,text='Core-Shell',variable=self.haveshell, value=True ,command = self.core_shell_label)
        
        self.core_material_combo = ttk.Combobox(root, values = material_files)
        self.core_material_combo.current(INITIAL_SETTING["NO"+str(self.num)+"_CORE_MATERIAL"].values[0])
        self.core_material_combo.bind("<<ComboboxSelected>>", self.core_material_callbackfunc)
        
        self.shell_material_combo = ttk.Combobox(root, values = material_files[0:-2])
        self.shell_material_combo.current(INITIAL_SETTING["NO"+str(self.num)+"_SHELL_MATERIAL"].values[0])
        self.shell_material_combo.bind("<<ComboboxSelected>>", self.shell_material_callbackfunc)
        
        self.custom_value1 = tkinter.Entry(width=WID)
        self.custom_value1.insert(tkinter.END,INITIAL_SETTING["NO"+str(self.num)+"_CUSTOM_VALUE1"].values[0])
        self.custom_value2 = tkinter.Entry(width=WID)
        self.custom_value2.insert(tkinter.END,INITIAL_SETTING["NO"+str(self.num)+"_CUSTOM_VALUE2"].values[0])
        self.custom_value3 = tkinter.Entry(width=WID)
        self.custom_value3.insert(tkinter.END,INITIAL_SETTING["NO"+str(self.num)+"_CUSTOM_VALUE3"].values[0])
        self.custom_value4 = tkinter.Entry(width=WID)
        self.custom_value4.insert(tkinter.END,INITIAL_SETTING["NO"+str(self.num)+"_CUSTOM_VALUE4"].values[0])
        
        self.import_file_combo = ttk.Combobox(root, values = custom_files,width=20)
        self.import_file_combo.current(INITIAL_SETTING["NO"+str(self.num)+"_IMPORT_FILE"].values[0])
        
        self.concentration   = tkinter.Entry(width=WID)
        self.concentration.insert(tkinter.END,INITIAL_SETTING["NO"+str(self.num)+"_CONCENTRATION"].values[0])
        self.core_diameter   = tkinter.Entry(width=WID)
        self.core_diameter.insert(tkinter.END,INITIAL_SETTING["NO"+str(self.num)+"_CORE_DIAMETER"].values[0])
        self.shell_thickness = tkinter.Entry(width=WID)
        self.shell_thickness.insert(tkinter.END,INITIAL_SETTING["NO"+str(self.num)+"_SHELL_THICKNESS"].values[0])
        
        self.conversion_button = tkinter.Button(text="Conversion",command = self.conversion)
        
        self.min_free_path      = tkinter.Entry(width=WID)
        self.min_free_path_wave = tkinter.Entry(width=WID)
        
    #Place widgets
    def place_widget(self):
        labeler(self.start_row,1,1," Particle No"+str(self.num)+" ##")
        labeler(self.start_row+1,3,1,"material")
        labeler(self.start_row+1,5,1,"           ")
        labeler(self.start_row+1,6,1,"           ")
        
        self.core_only_radio.grid(row=self.start_row+2,column=1)
        labeler(self.start_row+2,2,1,"Core")
        self.core_material_combo.grid(row=self.start_row+2,column=3)
        labeler(self.start_row+2,7,1,"diameter (nm)")
        self.core_diameter.grid(row=self.start_row+2, column=8)
        self.core_shell_radio.grid(row=self.start_row+3,column=1)
        labeler(self.start_row+4,2,1,"concentration (*10^9 num/mL)")
        self.concentration.grid(row=self.start_row+4, column=3)
        self.conversion_button.grid(row=self.start_row+4, column=4)
        labeler(self.start_row+4,5,2,"min mean free path (mm)")
        self.min_free_path.grid(row=self.start_row+4, column=7)
        labeler(self.start_row+4,8,1,"at wavelength (nm)")
        self.min_free_path_wave.grid(row=self.start_row+4, column=9)

    #Check all inputs for conversion from min_free_path to concentration
    def check_input(self):       
        flag = True
        if((self.min_free_path.get() == "") or(self.min_free_path.get() == 0)):
            flag = False
        if(self.core_diameter.get() == ""):
            flag = False
        #Core-only
        if(self.haveshell.get() == False):
            if(self.core_material_combo.get() == "Absorber - Constant Qabs"):
                if(self.custom_value1.get() == ""):
                    flag = False
            elif(self.core_material_combo.get() == "Absorber - Custom Qabs"):
                if(self.import_file_combo.get() == ""):
                    flag = False
            elif(self.core_material_combo.get() == "Constant"):
                if(self.custom_value1.get() == "" or self.custom_value2.get() == ""):
                    flag = False
        #Core-shell
        elif(self.haveshell.get() == True):
            if(self.shell_thickness.get() == ""):
                flag = False
            if(self.core_material_combo.get() == "Constant"):
                if(self.custom_value1.get() == "" or self.custom_value2.get() == ""):
                    flag = False
            if(self.shell_material_combo.get() == "Constant"):
                if(self.custom_value3.get() == "" or self.custom_value4.get() == ""):
                    flag = False
        return flag
        
    def core_material_callbackfunc(self,event):
        val = self.core_material_combo.get()
        #Core
        if(val == "Constant"):
            labeler(self.start_row+1,4,1,"     n     ")
            labeler(self.start_row+1,5,1,"     k     ")
            self.custom_value1.grid(row=self.start_row+2, column=4)
            self.custom_value2.grid(row=self.start_row+2, column=5)
            self.import_file_combo.grid_forget()
            
        elif(val ==  "Absorber - Constant Qabs"):
            labeler(self.start_row+1,4,1,"    Qabs   ")
            labeler(self.start_row+1,5,1,"           ")
            self.custom_value1.grid(row=self.start_row+2, column=4)
            self.custom_value2.grid_forget()
            self.import_file_combo.grid_forget()
            self.haveshell.set(0)
            self.delete_core_shell_label()

        elif(val == "Absorber - Custom Qabs"):
            labeler(self.start_row+1,4,1," import ")
            labeler(self.start_row+1,5,1,"             ")
            self.custom_value1.grid_forget()
            self.custom_value2.grid_forget()
            self.import_file_combo.grid(row=self.start_row+2,column=4,columnspan=3)
            self.haveshell.set(0)
            self.delete_core_shell_label()

        else:#import refractive index file
            if(self.shell_material_combo.get() != "Constant"):
                labeler(self.start_row+1,4,1,"            ")
                labeler(self.start_row+1,5,1,"            ")
            self.custom_value1.grid_forget()
            self.custom_value2.grid_forget()
            self.import_file_combo.grid_forget()

    def shell_material_callbackfunc(self,event):
        val = self.shell_material_combo.get()
        
        if(val == "Constant"):
            labeler(self.start_row+1,4,1,"     n     ")
            labeler(self.start_row+1,5,1,"     k     ")
            self.custom_value3.grid(row=self.start_row+3, column=4)
            self.custom_value4.grid(row=self.start_row+3, column=5)
            
        else:
            if(self.core_material_combo.get() != "Constant"):
                labeler(self.start_row+1,4,1,"            ")
                labeler(self.start_row+1,5,1,"            ")
            self.custom_value3.grid_forget()
            self.custom_value4.grid_forget()

    def core_shell_label(self):
        if((self.core_material_combo.get() ==  "Absorber - Constant Qabs") or (self.core_material_combo.get() == "Absorber - Custom Qabs")):
            self.haveshell.set(0)
        else:
            labeler(self.start_row+3,2,1,"Shell")
            self.shell_material_combo.grid(row=self.start_row+3,column=3)
            self.shell_thickness.grid(row=self.start_row+3, column=8)
            labeler(self.start_row+3,7,1,"thickness (nm)")

    def delete_core_shell_label(self):
        labeler(self.start_row+3,2,1,"            ")
        labeler(self.start_row+3,7,1,"                                     ")
        self.shell_material_combo.grid_forget()
        self.shell_thickness.grid_forget()
        self.custom_value3.grid_forget()
        self.custom_value4.grid_forget()
    
    def conversion(self):
        min_wave = round_sig(float(setting.min_wave.get()))
        max_wave = round_sig(float(setting.max_wave.get()))
        interval = round_sig(float(setting.interval_wave.get()))
        if(self.check_input()):
            all_eff = all_eff_dataframe(self)
            max_extinction_cross = 0
            max_wavelength = 0
            for wave in np.arange(min_wave,max_wave+interval,interval):
                extinction_eff = all_eff["Qext"][wave] 
                extinction_cross = float(self.core_diameter.get()) **2 * math.pi / 4 * extinction_eff
                if(extinction_cross > max_extinction_cross):
                    max_extinction_cross  = extinction_cross
                    max_wavelength = wave
            self.min_free_path_wave.delete(0, tkinter.END)
            self.min_free_path_wave.insert(tkinter.END,max_wavelength)
            self.concentration.delete(0, tkinter.END)
            self.concentration.insert(tkinter.END,round_sig(1 / float(self.min_free_path.get()) / max_extinction_cross * 1000000))
    
    def namer(self):
         #Core-only
        if(self.haveshell.get() == False):
            if(self.core_material_combo.get() == "Absorber - Constant Qabs"):
                return "Qabs="+self.custom_value1.get()
            elif(self.core_material_combo.get() == "Absorber - Custom Qabs"):
                return self.import_file_combo.get()[0:-4]
            elif(self.core_material_combo.get() == "Constant"):
                return self.custom_value1.get() +"+i" + self.custom_value2.get()
            else:
                return self.core_material_combo.get()[0:-4]
            
        #Core-shell
        elif(self.haveshell.get() == True):
            if(self.core_material_combo.get() == "Constant"):
                core_name = self.custom_value1.get() +"+i" + self.custom_value2.get()
            else:
                core_name = self.core_material_combo.get()[0:-4]
            if(self.shell_material_combo.get() == "Constant"):
                shell_name = self.custom_value3.get() +"+i" + self.custom_value4.get()
            else:
                shell_name = self.shell_material_combo.get()[0:-4]
            return core_name +"@"+shell_name

#simple label
def labeler(r,c,span,label):
    labe = tkinter.Label(text=label)
    labe.grid(row=r, column=c,columnspan = span, padx=5)
    
#return[Qsca,Qabs,Qeff]
def all_eff_dataframe(particle):
    Qeff = []
    waves = []
    min_wave = round_sig(float(setting.min_wave.get()))
    max_wave = round_sig(float(setting.max_wave.get()))
    interval = round_sig(float(setting.interval_wave.get()))
    nmedium = float(setting.host_refractive_index.get())
    for wave in np.arange(min_wave,max_wave+interval,interval):
        waves.append(wave)
    #Core-only
    if(particle.haveshell.get() == False):
        if(particle.core_material_combo.get() == "Absorber - Constant Qabs"):
            for wave in np.arange(min_wave,max_wave+interval,interval):
                Qeff.append([0,float(custom_value1_rec2.get()),float(custom_value1_rec2.get())])

        elif(particle.core_material_combo.get() == "Absorber - Custom Qabs"):
            Qabs = spline_custom(min_wave,max_wave,"custom/" + particle.import_file_combo.get(),2)
            for wave in np.arange(min_wave,max_wave+interval,interval):
                Qeff.append([0,Qabs["Qabs"][wave],Qabs["Qabs"][wave]])
                
        elif(particle.core_material_combo.get() == "Constant"):
            ref =  complex(float(particle.custom_value1.get()),float(particle.custom_value2.get()))
            res = ps.MieQ(ref,100,float(particle.core_diameter.get()),asDict=True,nMedium = nmedium)
            for wave in np.arange(min_wave,max_wave+interval,interval):
                Qeff.append([res["Qsca"],res["Qabs"],res["Qext"]])
            
        else:
            ref_df = spline(min_wave,max_wave,"material/" +particle.core_material_combo.get(),2)
            for wave in np.arange(min_wave,max_wave+interval,interval):
                ref  = complex(ref_df["n"][wave],ref_df["k"][wave])
                res = ps.MieQ(ref,wave,float(particle.core_diameter.get()),asDict=True,nMedium = nmedium)
                Qeff.append([res["Qsca"],res["Qabs"],res["Qext"]])
                
    #Core-shell
    elif(particle.haveshell.get() == True):
        if(particle.core_material_combo.get() != "Constant"):
            ref_core_df  = spline(min_wave,max_wave,"material/" + particle.core_material_combo.get(),2)
        if(particle.shell_material_combo.get() != "Constant"):
            ref_shell_df = spline(min_wave,max_wave,"material/" + particle.shell_material_combo.get(),2)
            
        for wave in np.arange(min_wave,max_wave+interval,interval):
            if(particle.core_material_combo.get() == "Constant"):
                ref_core = complex(float(particle.custom_value1.get()),float(particle.custom_value2.get()))
            else:
                ref_core = complex(ref_core_df["n"][wave],ref_core_df["k"][wave])
                
            if(particle.shell_material_combo.get() == "Constant"):
                ref_shell = complex(float(particle.custom_value3.get()),float(particle.custom_value4.get()))
            else:
                ref_shell = complex(ref_shell_df["n"][wave],ref_shell_df["k"][wave])
            outer_diameter = float(particle.shell_thickness.get()) * 2 + float(particle.core_diameter.get())
            res = ps.MieQCoreShell(ref_core / nmedium, ref_shell / nmedium, wave / nmedium, float(particle.core_diameter.get()),outer_diameter,asDict=True)
            if(type(res) == tuple):
                Qeff.append([res[1],res[2],res[0]])
            else:
                Qeff.append([res["Qsca"],res["Qabs"],res["Qext"]])
            
    return pd.DataFrame(np.array(Qeff),index = waves,columns = ["Qsca","Qabs","Qext"])

#np.array(angle_dependency) size = ANGLE_ARR_NUMBER
def angle_dataframe(particle,wavelength):
    min_wave = round_sig(float(setting.min_wave.get()))
    max_wave = round_sig(float(setting.max_wave.get()))
    interval = round_sig(float(setting.interval_wave.get()))
    #Core-only
    if(particle.haveshell.get() == False):
        if(particle.core_material_combo.get() == "Absorber - Constant Qabs"):
            return np.array([0 for i in range(ANGLE_ARR_NUMBER)])

        elif(particle.core_material_combo.get() == "Absorber - Custom Qabs"):
            return np.array([0 for i in range(ANGLE_ARR_NUMBER)])
                
        elif(particle.core_material_combo.get() == "Constant"):
            ref =  complex(float(particle.custom_value1.get()),float(particle.custom_value2.get()))
            return ps.ScatteringFunction(ref,wavelength,float(particle.core_diameter.get()),angularResolution = 0.1,nMedium = float(setting.host_refractive_index.get()))[3]
        else:
            ref_df = spline(round_sig(float(setting.min_wave.get())),round_sig(float(setting.max_wave.get())),"material/" + particle.core_material_combo.get() ,2)
            ref  = complex(ref_df["n"][wavelength],ref_df["k"][wavelength])
            return ps.ScatteringFunction(ref,wavelength,float(particle.core_diameter.get()),angularResolution = 0.1,nMedium = float(setting.host_refractive_index.get()))[3]
                
    #Core-shell
    elif(particle.haveshell.get() == True):
        if(particle.core_material_combo.get() != "Constant"):
            ref_core_df  = spline(round_sig(float(setting.min_wave.get())),round_sig(float(setting.max_wave.get())),"material/" + particle.core_material_combo.get() ,2)
        if(particle.shell_material_combo.get() != "Constant"):
            ref_shell_df = spline(round_sig(float(setting.min_wave.get())),round_sig(float(setting.max_wave.get())),"material/" + particle.shell_material_combo.get(),2)
            
        if(particle.core_material_combo.get() == "Constant"):
            ref_core = complex(float(particle.custom_value1.get()),float(particle.custom_value2.get()))
        else:
            ref_core = complex(ref_core_df["n"][wavelength],ref_core_df["k"][wavelength])

        if(particle.shell_material_combo.get() == "Constant"):
            ref_shell = complex(float(particle.custom_value3.get()),float(particle.custom_value4.get()))
        else:
            ref_shell = complex(ref_shell_df["n"][wavelength],ref_shell_df["k"][wavelength])
        outer_diameter = float(particle.shell_thickness.get()) * 2 + float(particle.core_diameter.get())
        return ps.CoreShellScatteringFunction(ref_core,ref_shell,wavelength,float(particle.core_dimaeter.get()),outer_diameter,angularResolution=0.1)[3]
                
#return material and custom files arr
def get_files():
    all_material_files = os.listdir("material")
    material_files = []
    for i in range(len(all_material_files)):
        if(".csv" in all_material_files[i] or ".txt" in all_material_files[i]):
             material_files.append(all_material_files[i])
                
    all_custom_files = os.listdir("custom")
    custom_files = []
    for i in range(len(all_custom_files)):
        if(".csv" in all_custom_files[i] or ".txt" in all_custom_files[i]):
             custom_files.append(all_custom_files[i])
                
    material_files.append("Constant")
    material_files.append("Absorber - Constant Qabs")
    material_files.append("Absorber - Custom Qabs")
    
    return [material_files,custom_files]

#interpolate import file
def spline(min_wave,max_wave,file,columns):
    interval = round_sig(float(setting.interval_wave.get()))
    lbd0 = np.arange(round_sig(float(setting.min_wave.get())),round_sig(float(setting.max_wave.get()))+interval,interval)
    if(file[-1] == "t"):
        deli='\t'
    if(file[-1] == "v"):
        deli=','   
    rawdata = np.loadtxt(file,delimiter=deli, comments='%',skiprows=1)  # delimiter='\t' may not be needed
    values = []
    for i in range(1,columns+1):
        interp = interpolate.interp1d(rawdata[:,0],rawdata[:,i],kind="cubic")
        values.append(interp(lbd0))
    
    if(columns == 1):
        return pd.DataFrame(np.array(values).T,index = lbd0,columns = ["Qabs"])
    elif(columns == 2):
        return pd.DataFrame(np.array(values).T,index = lbd0,columns = ["n","k"])
    else:
        print("imported file has too much columns")
        
#r=1
def spherical2cartesian(position):
    newPosition = [0,0,0]
    newPosition[0] = np.sin(position[0]) * np.cos(position[1])
    newPosition[1] = np.sin(position[0]) * np.sin(position[1])
    newPosition[2] = np.cos(position[0])
    return newPosition

#r=1
def cartesian2spherical(position):
    newPosition = np.empty([2], dtype=np.float64)
    newPosition[0] = np.arccos(position[2])
    newPosition[1] = np.arctan2(position[1], position[0])
    nan_index = np.isnan(newPosition[0])
    newPosition[nan_index,1] = 0
    return [newPosition[0],newPosition[1]]

#x_rotation_matrix
def rotate_x(radian):
    return np.matrix([[1,0,0],[0,np.cos(radian),-1*np.sin(radian)],[0,np.sin(radian),np.cos(radian)]])
#y_rotation_matrix
def rotate_y(radian):
    return np.matrix([[np.cos(radian),0,np.sin(radian)],[0,1,0],[-1*np.sin(radian),0,np.cos(radian)]])
#z_rotation_matrix
def rotate_z(radian):
    return np.matrix([[np.cos(radian),-1*np.sin(radian),0],[np.sin(radian),np.cos(radian),0],[0,0,1]])

#rotation matrix from inpur vector to [0,0,1]
def rotate2z_matrix(vector):
    a = vector / np.linalg.norm(vector)
    b = np.array([a[0],0,a[2]])
    c = b / np.linalg.norm(b)
    d = np.array(np.dot(rotate_y(-1*np.sign(c[0])*math.acos(c[2])),a))[0]
    if(d[2] > 1):
        d[2] = 1
    if(d[2] < -1):
        d[2] = -1
    e = np.array(np.dot(rotate_x(np.sign(d[1])*np.sign(d[2])*math.acos(d[2])),d))[0]
    return np.dot(rotate_x(np.sign(d[1])*np.sign(d[2])*math.acos(d[2]))    ,   rotate_y(-1*np.sign(c[0])*math.acos(c[2])) )

#incident photons position
def incident_posi():
    flux = np.empty((0,3), int)
    for i in np.linspace(-1* setting.block_xy/2,setting.block_xy/2, 1000):
        for j in np.linspace(-1*setting.block_xy/2, setting.block_xy/2, 1000):
            flux = np.append(flux,np.array([[i,j, -1 * float(setting.block_z.get())*1000/2]]),axis = 0)
    return flux

#incident photons position
def incident_posi2():
    flux = np.empty((0,3), int)
    for i in np.linspace(-1*float(setting.block_x.get())*1000/2,float(setting.block_x.get())*1000/2,100):
        for j in np.linspace(-1*float(setting.block_y.get())*1000/2, float(setting.block_y.get())*1000/2, 250 ):
            flux = np.append(flux,np.array([[i,j, -1 * float(setting.block_z.get())*1000/2]]),axis = 0)
    return flux

#create randam vector scattering isotropic
def scattering_isotropic():
    scattering=np.array([np.random.rand()-0.5,np.random.rand()-0.5,np.random.rand()-0.5])
    return  np.array([scattering / np.linalg.norm(scattering)])
    
#place after appling periodic boundary condition
def periodic_boudary(place):
    block_half_x,block_half_y = float(setting.block_x.get())*1000/2,float(setting.block_y.get())*1000/2
    while(True):
        if(place[0] < -1* block_half_x):
            place[0] = place[0] + 2 * block_half_x
        if(place[0] >   block_half_x):
            place[0] = place[0] - 2 * block_half_x
        
        if(place[1] < -1* block_half_y):
            place[1] = place[1] + 2 * block_half_y
            
        if(place[1] >  block_half_y):
            place[1] = place[1] - 2 * block_half_y
            
        if((place[0]>=-1*block_half_x)and(place[0]<=block_half_x)and(place[1] >= -1 * block_half_y)and(place[1] <= block_half_y)):
            return place

#return photon's place(PERIODIC BOUNDARY MODE)
def escape_judge_periodic(place):
    place = periodic_boudary(place)
    if(place[2] > float(setting.block_z.get())*1000/2):
        return FORWARD_SCATTERING
    if(place[2] < -1 * float(setting.block_z.get())*1000/2):
        return BACKWARD_SCATTERING
    return IN_SYSTEM

#return photon's place(not PERIODIC BOUNDARY MODE)
def escape_judge(origin,vector):
    if(
        (collision_x < block_half_x)  and (collision_x > -1*block_half_x)
        and(collision_y<block_half_y) and (collision_y > -1*block_half_y)
        and(collision_z<block_half_z) and (collision_z > -1*block_half_z)
    ):
        return IN_SYSTEM
    
    if(vector[0] == 0):
        escape_time_x = 100000000000
    else:
        escape_time_x = (np.sign(vector[0]) *  float(setting.block_x.get())*1000/2 - origin[0]) / vector[0]
    if(vector[1] == 0):
        escape_time_y = 100000000000
    else:
        escape_time_y = (np.sign(vector[1]) *  float(setting.block_y.get())*1000/2 - origin[1]) / vector[1]
    if(vector[2] == 0):
        escape_time_z = 100000000000
    else:
        escape_time_z = (np.sign(vector[2]) *  float(setting.block_z.get())*1000/2 - origin[2]) / vector[2]
    if((escape_time_z < escape_time_x)and (escape_time_z < escape_time_y)):
        if(vector[2] > 0):
            return FORWARD_SCATTERING
        else:
            return BACKWARD_SCATTERING
    if(escape_time_x < escape_time_y):
        return SIDEWARD_SCATTERING_X
    else:
        return SIDEWARD_SCATTERING_Y

def getNearestValue(list, num):    
    idx = np.abs(np.asarray(list) - num).argmin()
    return idx

def result_file_write_columns(result_file):
    particle_types = int(setting.particle_types.get()) 
    with open(result_file, mode='a') as result:
        #result file
        result.write("wavelength,transmission,scattering,absorption,mean_events,mean_optical_path(um),mean_free_path(um),,")
        result.write("forward_scattering,forward_scattering_events,forward_scattering_multiple_rate,mean_forward_scattering_optical_path(um),,")
        result.write("backword_scattering,backward_scattering_events,backward_scattering_multiple_rate,mean_backward_scattering_optical_path(um),,")
        result.write("sideword_scattering,sideward_scattering_events,sideward_scattering_multiple_rate,mean_sideward_scattering_optical_path(um),,")
        result.write("absorption,absorption_events,absorption_multiple_rate,mean_absorption_optical_path(um),,")
        for particle in range(particle_types):
            result.write(particles[particle].namer()+"_mean_free_path(um),")
        for particle in range(particle_types):
            result.write(particles[particle].namer()+"_collision_rate,")
        for particle in range(particle_types):
            result.write(particles[particle].namer()+"_collision_events_per_photon,")
        result.write(",,SIMULATION_SETTING,")
        for particle in range(particle_types):
            result.write(particles[particle].namer()+"Core_diameter(nm):"+particles[particle].core_diameter.get()+"   Shell_thickness(nm):"+particles[particle].shell_thickness.get())
            result.write("    Concentration(*10^9 number/ml):" + particles[particle].concentration.get() + ",")
        result.write(",block_x(um):"+setting.block_x.get()+",block_y(um):"+setting.block_y.get()+",block_z(um):"+setting.block_z.get())
        result.write(",incident_photons:"+ str(int(setting.incident_photons.get()))+",host_refractive_index:"+setting.host_refractive_index.get())
        result.write(",random_seed:"+str(RANDOM_SEED))
        result.write(",PERIODIC BOUNDARY MODE:"+str(setting.PERIODIC_BOUNDARY_MODE.get()))
        result.write(",LOG_MODE:"+str(setting.LOG_MODE.get()))
        result.write("\n")

def writer(shen,result_file):
    with open(result_file, mode='a') as result:
        for i in range(len(shen)):
            result.write(str(shen[i]) + ",")

#return [photons end,optical path,[each particle scattering events]]
def photon(num,angles,sca_rates,cross_section_densities,total_cross_section_density,log_file):
    LOG_MODE = setting.LOG_MODE.get()
    PERIODIC_BOUNDARY_MODE = setting.PERIODIC_BOUNDARY_MODE.get()
    block_half_x,block_half_y,block_half_z = float(setting.block_x.get())*1000/2,float(setting.block_y.get())*1000/2,float(setting.block_z.get())*1000/2
    generation = 0
    collision_x,collision_y,collision_z = 0,0,0
    origin_x,origin_y = 0,0
    origin_z = -1 * block_half_z
    vector_x,vector_y,vector_z = 0,0,1
    path = 0
    total_path = 0
    material_events = [0 for i in range(int(setting.particle_types.get()))]

    while(True):#loop until photon death
        free_path = random.expovariate(total_cross_section_density)
        #new place
        collision_x = origin_x + vector_x * free_path
        collision_y = origin_y + vector_y * free_path
        collision_z = origin_z + vector_z * free_path
        if(LOG_MODE):
            with open(log_file, mode='a') as log:
                log.write(str(num)+"," +str(round_sig(free_path/1000)) + "," + str(round_sig(origin_x/1000))+","+str(round_sig(origin_y/1000))+",")
                log.write(str(round_sig(origin_z/1000))+",")
                log.write(str(round_sig(collision_x/1000))+","+str(round_sig(collision_y/1000))+","+str(round_sig(collision_z/1000)) + ",")
                log.write(str(round_sig(vector_x)) + "," + str(round_sig(vector_y)) + ","+str(round_sig(vector_z)) + ",")

        if(PERIODIC_BOUNDARY_MODE):
            judge = escape_judge_periodic([collision_x,collision_y,collision_z]) 
        ##not periodic boundary mode
        else:
            judge = escape_judge([origin_x,origin_y,origin_z],[vector_x,vector_y,vector_z])

        #forward
        if(judge == FORWARD_SCATTERING):
            if(generation == 0):
                total_path = block_half_z*2
                if(LOG_MODE):
                    with open(log_file, mode='a') as log:
                        log.write("None,None,transmission\n")
                return [TRANSMISSION,total_path,generation,material_events]
            else:
                #angle[getNearestValue(angle_arr,cartesian2spherical([vector_x,vector_y,vector_z])[0])] += 1 
                total_path = path+(block_half_z - origin_z)  /  vector_z-free_path 
                if(LOG_MODE):
                    with open(log_file, mode='a') as log:
                        log.write("None,None,forward_scattering\n")
                #0:transmission、1:forward scattering 2:sideward scattering 3:backward scattering 4:absorption
                return [FORWARD_SCATTERING,total_path,generation,material_events]

        #backward scattering
        elif(judge == BACKWARD_SCATTERING):
            #angle[getNearestValue(angle_arr,cartesian2spherical([vector_x,vector_y,vector_z])[0])] += 1
            total_path = path + (-1*block_half_z - origin_z)  /  vector_z - free_path #計算式

            if(LOG_MODE):
                with open(log_file, mode='a') as log:
                    log.write("None,None,backward_scattering\n")
            return [BACKWARD_SCATTERING,total_path,generation,material_events]

        #sideward scattering 
        elif((judge == SIDEWARD_SCATTERING_X) or (judge == SIDEWARD_SCATTERING_X)):
            if(judge ==  SIDEWARD_SCATTERING_X):       
                total_path = path + (np.sign(vector_x)*block_half_x - origin_x)  /  vector_x - free_path #計算式
            if(judge ==  SIDEWARD_SCATTERING_Y):       
                total_path = path + (np.sign(vector_y)*block_half_y - origin_y)  /  vector_y - free_path #計算式
            if(LOG_MODE):
                with open(log_file, mode='a') as log:
                    log.write("None,None,sideward_scattering\n")
            return [SIDEWARD_SCATTERING,total_path,generation,material_events]

        #photon still in system
        elif(judge == IN_SYSTEM):
            collision_particle = random.choices(range(int(setting.particle_types.get())),weights = cross_section_densities)[0]
            sca_rate = sca_rates[collision_particle]
            material_events[collision_particle]+=1
            #scattering
            if(np.random.rand() < sca_rate):
                sca_vector = spherical2cartesian([random.choices(ANGLE_ARR,weights=angles[collision_particle])[0],(random.random()-0.5)*2 * math.pi])
                original_angle_cartesian = np.array([vector_x,vector_y,vector_z])
                vector_x,vector_y,vector_z = np.array(np.dot(rotate2z_matrix(original_angle_cartesian),sca_vector))[0]
                if(LOG_MODE):
                    with open(log_file, mode='a') as log:
                        log.write(particles[collision_particle].namer() + ",scattering\n")
            #absorption
            else:
                total_path += path
                if(LOG_MODE):
                    with open(log_file, mode='a') as log:
                        log.write(particles[collision_particle].namer() + ",absorption,absorption\n")
                return [ABSORPTION,total_path,generation+1,material_events]
            generation += 1
            origin_x = collision_x
            origin_y = collision_y
            origin_z = collision_z
            path += free_path
            
def start():
    """
    Initial Settings
    """
    ##############################################Save INITIAL_SETTING###########################################################
    save_initial_setting()
    np.random.seed(RANDOM_SEED)
    ##############################################Result file directory###########################################################
    result_path = "montecarlo/"
    #####Make folder
    if(os.path.exists(result_path[0:-1])==False):
        os.mkdir(result_path[0:-1])
    ##################################################LOG_MODE##############################################################
    LOG_MODE = setting.LOG_MODE.get()
    ############################################number of cpu (when you try multicore process)##############################################################
    number_of_cpu = 4
    if(LOG_MODE):
        number_of_cpu = 1
    ##################################################num##########################################################
    incident_photons = int(setting.incident_photons.get()) 
    flux_power = 1 / incident_photons #power per one photon
    #################################################wavelength setting#############################################################
    min_wave = round_sig(float(setting.min_wave.get()))
    max_wave = round_sig(float(setting.max_wave.get()))
    interval = round_sig(float(setting.interval_wave.get()))
    ##############################################number of particle types###########################################################
    particle_types = int(setting.particle_types.get()) 
    ##############################################################################################################
    all_eff = []
    diameters = []
    concentrations = []
    for particle in range(particle_types):
        if(particles[particle].haveshell.get()):
            diameters.append(float(particles[particle].core_diameter.get()) + 2 * float(particles[particle].shell_thickness.get()))
        else:
            diameters.append(float(particles[particle].core_diameter.get()))
        concentrations.append(float(particles[particle].concentration.get())  * 10**9 * 10**(-21))
        all_eff.append(all_eff_dataframe(particles[particle]))
    
    result_file = result_path+"test.csv"
    result = open(result_file,"w")
    result.close()
    result_file_write_columns(result_file)        

    """
    start calculating
    """
    for wavelength in np.arange(min_wave,max_wave+interval,interval):
        angles = []
        sca_rates = []
        cross_section_densities = []
        log_file = result_file[0:-4] + "_log_at"+str(wavelength)+"nm.csv"
        if(LOG_MODE):
            log = open(log_file,"w")
            with open(log_file, mode='a') as log:
                log.write("Photon number,free path(um),origin_x(um),origin_y(um),origin_z(um),end_x(um),end_y(um),end_z(um),")       
                log.write("vector_x,vector_y,vector_z,collision_particle,phenomena,photon death by,\n")
        for particle in range(particle_types):
            angles.append(angle_dataframe(particles[particle],wavelength))
            sca_rates.append(all_eff[particle]["Qsca"][wavelength] / all_eff[particle]["Qext"][wavelength])
            cross_section_densities.append(all_eff[particle]["Qext"][wavelength]*diameters[particle]**2*math.pi /4* concentrations[particle])
        
        total_cross_section_density = sum(cross_section_densities)
        
        boundary_times = 0
        result_list = []
        iterator = range(incident_photons),angles,sca_rates,cross_section_densities,total_cross_section_density,log_file
        #################################not good now(you can try if you want to calculate faster)########################################
        #with Pool(processes = number_of_cpu) as p:
        #    print("ds")
        #    result_list.append(p.map(func = photon,iterable = iterator))
        #result = result_list[0]
        for i in range(incident_photons):
             result_list.append(photon(i,angles,sca_rates,cross_section_densities,total_cross_section_density,log_file))
        result = result_list
        total_events = 0
        transmission = 0
        total_path = 0
        forward_sca,  forward_events, forward_mean_path, forward_multiple_rate, forward_number = 0,0,0,0,0
        sideward_sca,sideward_events,sideward_mean_path,sideward_multiple_rate,sideward_number = 0,0,0,0,0
        backward_sca,backward_events,backward_mean_path,backward_multiple_rate,backward_number = 0,0,0,0,0
        absorption,absorption_events,absorption_mean_path,absorption_multiple_rate,absorption_number = 0,0,0,0,0
        material_events = [0 for i in range(int(setting.particle_types.get()))]
        
        #[photon destiny,optical path,events,events of each material]
        for i in range(incident_photons):
            total_path += result[i][1]
            total_events += result[i][2]
            for material in range(int(setting.particle_types.get())):
                material_events[material] += result[i][3][material]

            if(result[i][0] == TRANSMISSION):#transmission
                transmission+=flux_power
                
            elif(result[i][0] == FORWARD_SCATTERING):#forward
                forward_number += 1
                forward_sca +=flux_power
                forward_events += result[i][2]
                forward_mean_path += result[i][1]
                if(result[i][2] >= 2):
                    forward_multiple_rate += 1

            elif(result[i][0] == SIDEWARD_SCATTERING):#sideward
                sideward_number += 1
                sideward_sca +=flux_power
                sideward_events += result[i][2]
                sideward_mean_path += result[i][1]
                if(result[i][2] >= 2):
                    sideward_multiple_rate += 1

            elif(result[i][0] == BACKWARD_SCATTERING):#backscattering
                backward_number += 1
                backward_sca += flux_power
                backward_events += result[i][2]
                backward_mean_path += result[i][1]
                if(result[i][2] >= 2):
                    backward_multiple_rate += 1

            elif(result[i][0] == ABSORPTION):#absorption
                absorption_number += 1
                absorption += flux_power
                absorption_events += result[i][2]
                absorption_mean_path += result[i][1]
                if(result[i][2] >= 2):
                    absorption_multiple_rate += 1
                    
        #calculate each parameters
        mean_total_events = round_sig(total_events / incident_photons)
        mean_total_path = round_sig(total_path / (incident_photons*1000))
        mean_free_path = round_sig(1 / (total_cross_section_density * 1000))
        if(forward_number > 0):
            forward_events = round_sig(forward_events/forward_number)
            forward_mean_path = round_sig(forward_mean_path/forward_number/1000)
            forward_multiple_rate =round_sig(forward_multiple_rate/forward_number)
        if(sideward_number > 0):
            sideward_events = round_sig(sideward_events / sideward_number)
            sideward_mean_path = round_sig(sideward_mean_path /sideward_number/1000)
            sideward_multiple_rate = round_sig(sideward_multiple_rate/sideward_number)
        if(backward_number > 0):
            backward_events = round_sig(backward_events/backward_number)
            backward_mean_path = round_sig(backward_mean_path/backward_number/1000)
            backward_multiple_rate = round_sig(backward_multiple_rate/backward_number)
        if(absorption_number > 0):
            absorption_events = round_sig(absorption_events/absorption_number)
            absorption_mean_path = round_sig(absorption_mean_path/absorption_number/1000)
            absorption_multiple_rate = round_sig(absorption_multiple_rate/absorption_number)
      
        writer([wavelength,transmission,forward_sca+sideward_sca+backward_sca,absorption,mean_total_events,mean_total_path,mean_free_path,""],result_file)
        writer([ forward_sca, forward_events, forward_multiple_rate, forward_mean_path,""],result_file)
        writer([backward_sca,backward_events,backward_multiple_rate,backward_mean_path,""],result_file)
        writer([sideward_sca,sideward_events,sideward_multiple_rate,sideward_mean_path,""],result_file)
        writer([absorption, absorption_events,absorption_multiple_rate,absorption_mean_path,""],result_file)
        
        for particle in range(particle_types):
            writer([round_sig(1 / (cross_section_densities[particle]*1000))] , result_file)
        for particle in range(particle_types):
            if(sum(material_events) > 0):
                writer([round_sig(material_events[particle]/sum(material_events))] , result_file)
            else:
                writer([0] , result_file)
        for particle in range(particle_types):
            writer([round_sig(material_events[particle]/incident_photons)] , result_file)
        
        with open(result_file, mode='a') as result:
            result.write("\n")
    print("calculate ends")
    
def quit():
    root.destroy()
    exit()

material_files,custom_files = get_files()
root = tkinter.Tk()
root.geometry("1300x800")
root.title("Python GUI")

setting = Setting(master = root)
setting.start_row =  0
setting.particle_types()
particles = []
for num in range(MAX_PARICLE_TYPES):
    thisparticle = ParticleInfo(master=root)
    thisparticle.start_row = 4+9*num
    thisparticle.num = num + 1
    particles.append(thisparticle)
    particles[num].define_widget()
    particles[num].place_widget()
    
setting.start_row =   MAX_PARICLE_TYPES*8+8 #max_particles*8+8
setting.simulation()
root.mainloop()

print("END")

SAVED SETTING
calculate ends
END


In [11]:
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd
import math
import os
import random
from scipy import interpolate
import copy
from multiprocessing import Pool
import PyMieScatt as ps
import tkinter
from tkinter import ttk
import inspect

a = pd.read_csv("setting/initial_setting.csv",index_col = 0)
a.to_csv("test.csv")